In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from UCroma import PretrainedCROMA
import copy

from main import CRLoader as Loader, FExtractor, Trainer, Tester

sample_size = 120
batch_size = 16

Croma = PretrainedCROMA(pretrained_path='CR.pt', size='base', modality='both', image_resolution=sample_size)
num_features = Croma(SAR_images=torch.randn(1, 2, sample_size, sample_size), 
                     optical_images=torch.randn(1, 12, sample_size, sample_size))['joint_GAP'].shape[1]
head = nn.Sequential(nn.Linear(num_features, 1000), nn.ReLU(), nn.Linear(1000, 1000), nn.ReLU(), nn.Linear(1000, 2))

In [ ]:
it = 5
metrics = []

for i in range(it):
    
    loaders = []
    for n in range(1, 5):
        processor = Loader(opt_root_dir=f"/home/stagiaire/D/D/patchs/{n}n", sar_root_dir=f"/home/stagiaire/D/D/patchs/{n}Rn", num_folds=1)
        processor.load_data(sample_size=sample_size, batch_size=batch_size)
        loader = processor.loaders[0]
        feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
        features, labels = feature_extractor.extract_features()
        dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels))
        loaders.append(DataLoader(dataset, batch_size=batch_size, shuffle=True))  
            
    head_i = copy.deepcopy(head)
        
    train_loaders = [loaders[2],loaders[3],loaders[1]]
    test_loader = loaders[0]
        
    for train_loader in train_loaders:
        trainer = Trainer(head_i, train_loader)
        trainer.train(num_epochs=100)

    tester = Tester(head_i, test_loader, processor.combined_dataset)
    tester.evaluate()
    metrics.append(tester.report)

In [ ]:
from R import RGenerator

report = RGenerator(metrics).report()